### Mean shift for object tracking

In [47]:
import cv2
import numpy as np

video_file_path = 'videos/traffic_car_video.mp4'
cap = cv2.VideoCapture(0) # just pass the video file path here

ret, frame = cap.read()

# declare the initial box for tracking
track_box = (500,0,50,50) 
    
# convert the color frame to hsv 
hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
# create the mask using hsv color space 
# this code developed to track a red color ball 
# use webcam and show red ball from the green frame it start tracking
# required to optimize so that it automaticall detect the red ball at starting

mask1 = cv2.inRange(hsv_img, np.array([0,0,0]), np.array([15,255,255]))
mask2 = cv2.inRange(hsv_img, np.array([170,0,0]), np.array([180,255,255]))

mask = mask1 + mask2
    
# calculate the color histogram and normalize it    
img_hist = cv2.calcHist([hsv_img], [0], mask, [180], [0, 180])
cv2.normalize(img_hist, img_hist, 0, 255, cv2.NORM_MINMAX)
    
# setup the termination criteria
termination_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)


while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Not able to load video frame")
        break
        
    hsv_img2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    final_img = cv2.calcBackProject([hsv_img2], [0], img_hist, [0, 180], 1)
    
    ret, track_box = cv2.meanShift(final_img, track_box, termination_criteria)
    
    x,y,w,h = track_box
    
    img2 = cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 1)    
        
    img3 = cv2.flip(img2, 1)
    cv2.imshow('Video', img3)
    
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
    